# Example 4: Optimize a noisy multivariate system

Bayesian optimization also works for stochastic response functions. Here we illustrate this for a noisy multivariate system by determining the maximal value.

## Framework approach

We will solve the problem in two different ways:
1. using the closed-loop approach of the `.auto`-method
2. using the iterative optimization approach of the framework which requires using the methods `.ask` and `.tell`. This approach allows for iterative optimization and optimization of any callable function, known or otherwise.

The optimization process can be stopped after any number of iterations.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

### Get started: Import libraries

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/

pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

# Install creative_project from github (will require authentication with password)
pip install --user https://github.com/svedel/kre8_core/

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

### Set up the problem to be solved

Here create a noisy multivariate function defined below for the input vector $\mathbf{x} = (x_0, x_1)$

$$
f(\mathbf{x}) = - \frac{ (6 x_0 -2)^2 (6 x_1 -2)^2 \, \sin{(12 x_0- 4)} \sin{(12 x_1 - 4)} }{250} + \frac{1}{2 \sigma^2 \pi} \mathrm{e}^{- \left(\frac{x_0 - 0.5}{\sigma} \right)^2 - \left( \frac{x_1 - 0.5}{\sigma} \right)^2 }   + \xi \quad , \quad x_i \in [0; 1], \ i=0,1
$$

where $\xi$ is random number drawn from a uniform distribution (range $[0; 1]$) and $\sigma = 0.1$. This function has its average global maximum at $\mathbf{x}^* = (0.5,0.5)$.


In [ ]:
# define the function
def f2_dup(x,y):
    sigma = torch.tensor(0.1, dtype=torch.double)
    return (-(6 * x - 2) ** 2 * torch.sin(12 * x - 4))*(-(6 * y - 2) ** 2 * torch.sin(12 * y - 4))/250 + 1/torch.sqrt(2*sigma**2*np.pi) * torch.exp(-((x-0.5)/sigma)**2 - ((y-0.5)/sigma)**2 ) + torch.rand(x.size())

x = torch.linspace(0,1)
x0_plot, x1_plot = torch.meshgrid(x, x)
x_mg = torch.cat((torch.reshape(x0_plot, (10000, 1)), torch.reshape(x1_plot, (10000, 1))), dim=1)
resp = f2_dup(x0_plot, x1_plot)


fig = plt.figure(figsize=(8, 8))
ax = fig.gca(projection='3d')

surf = ax.plot_surface(x0_plot.numpy(), x1_plot.numpy(), resp.numpy())
plt.show()

Define the range for the covariates

In [ ]:
# define the range of interest
x0_init = 0.2
x1_init = 0.8
covars2d = [(x0_init, 0, 1), (x1_init, 0, 1)]

### Solution 1: Closed-loop solution approach using `.auto` method

Instantiate the `CreativeProject` class and solve the problem

In [ ]:
# initialize class instance
cc = CreativeProject(covars=covars2d)

# number of iterations
max_iter = 20

# run the auto-method
cc.auto(response_samp_func=f2_dup, max_iter=max_iter)

**PLOT THE PATH TO OPTIMALITY**

The best solution

In [ ]:
# run current_best method
cc.current_best()

### Solution 2: Iterative solution using `.ask` and `.tell` methods

Instantiate the `CreativeProject` class and solve the problem. In this case, we need to write our own loop to iterate. Notice that the `covars` and `response` variables are converted to `torch` tensors of size $1 \times \mathrm{\#covariates}$ to store them in the instantiated class, where they are used for retraining the model at each iteration.

In [ ]:
# initialize the class instance
cc2 = CreativeProject(covars=covars2d)

# run the solution
for i in range(max_iter):
    # generate candidate
    cc2.ask()

    # sample response
    covars = torch.tensor([[it.item() for it in cc2.proposed_X[-1]]], dtype=torch.double)
    response = torch.tensor([[f2_dup(cc2.proposed_X[-1]).item()]], dtype=torch.double)

    # report response
    cc2.tell(covars=covars, response=response)

Best guess after solving

In [ ]:
# run current_best method
cc2.current_best()